# Validate Monday.com tasks for integration issues

In [ ]:
#%load_ext nb_black

import logging

import pandas as pd

import prefect
from prefect import task, Flow, Parameter, unmapped
from prefect.executors import LocalExecutor, LocalDaskExecutor
from prefect.utilities.logging import get_logger

from datetime import timedelta, datetime
from box import Box

from mondaydotcom_utils.formatted_value import FormattedValue, get_col_defs
from mondaydotcom_utils.time_block import TimeBlock
from mondaydotcom_utils.utilities import validate_task_record, get_items_by_board

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient
from monday.resources.base import BaseResource

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
TASKS_BOARD_ID = "1883170887"

MONDAY_KEY = ""
environment = "dev"

In [ ]:
@task
def get_monday_client(key):
    logger = prefect.context.get("logger")
    conn = MondayClient(key)
    logger.info("Monday.com client created.")

    return conn

In [ ]:
@task
def get_users(monday_key):

    logger = prefect.context.get("logger")
    # bug between ProdPerfect and MDC's API: https://github.com/ProdPerfect/monday/issues/57
    query = """query
        {
            users () {
                id
                name
                email
                enabled
            }
        }"""
    
    base_resource = BaseResource(monday_key)
    users = base_resource._query(query)["data"]["users"]

    users_df = pd.DataFrame(users).set_index("id")
    return users_df

In [ ]:
def breakout_time_sessions(row):
    """
    Break down the Monday.com time structure into something simpler for us.

    This is used with a DataFrame.apply()
    """

    mct = TimeBlock()
    mct.parse(row["Actual Time"])
    return mct.total_duration_hours, mct.time_records

In [ ]:
@task
def apply_time_session_breakout(tasks_df):
    logger = prefect.context.get("logger")

    tasks_df[["Total Duration Hours", "Time Sessions"]] = tasks_df.apply(
        breakout_time_sessions, axis=1, result_type="expand"
    )
    return tasks_df.reset_index()

In [ ]:
@task
def validate_tasks(tasks_df):
    logger = prefect.context.get("logger")

    records = tasks_df.reset_index().to_dict("records")

    vald_recs = []

    for record in records:
        # break the record out into N records
        vald_rec = validate_task_record(record)
        if vald_rec:
            vald_recs.append(vald_rec)

    df = pd.DataFrame(vald_recs).set_index("index")
    return df

In [ ]:
@task
def get_tasks(monday_conn, get_only_done=False):

    logger = prefect.context.get("logger")

    if get_only_done:
        tasks_df = get_items_by_board(TASKS_BOARD_ID, "status", "Done")
    else:
        tasks_df = get_items_by_board(monday_conn, TASKS_BOARD_ID)

    # Do not include Posted tasks
    tasks_df = tasks_df.loc[
        ~tasks_df["Integration Message"].str.startswith("Posted", na=False)
    ]

    return tasks_df

In [ ]:
@task
def get_df_as_records(df):
    """Helpful for setting up the collection for mapping."""
    logger = prefect.context.get("logger")

    return df.to_dict("records")

In [ ]:
@task(max_retries=3, retry_delay=timedelta(seconds=15))
def update_task_integration_status(monday_conn, record):
    logger = prefect.context.get("logger")
    logger.debug(f"Updating Monday.com record for {record['Title']}")
    monday_conn.items.change_item_value(
        TASKS_BOARD_ID,
        record["monday_id"],
        "text01",
        f"{record['integration_state_rule']} - {datetime.now()}",
    )

In [ ]:
@task
def display_df(df, title, count=5):
    logger = prefect.context.get("logger")
    logger.info(title)
    display(df.head(count))

In [ ]:
with Flow("monday.com task integration") as flow:

    key = Parameter("key")

    conn = get_monday_client(key)

    # get users
    users_df = get_users(key)

    # get tasks
    task_items_df = get_tasks(conn)
    task_items_df_with_brokedown_time = apply_time_session_breakout(task_items_df)

    # validate against the actual time, session and owner count rules
    validated_tasks_df = validate_tasks(task_items_df_with_brokedown_time)

    # send updates back to Monday.com... this is all one-way so no reduce required
    vald_items = get_df_as_records(validated_tasks_df)
    update_task_integration_status.map(unmapped(conn), vald_items)

In [ ]:
if not MONDAY_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    MONDAY_KEY = secrets.apps.monday.API_KEY

params = {"key": MONDAY_KEY}
state = flow.run(parameters=params, executor=LocalDaskExecutor())